# Bank Customer Churn


## Data Background

This dataset can be found [here](https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset) on kaggle.com.  It appears to be fictitious data created for a ML classification exercise

## Data Dictionary


* **customer_id** - unused variable.
* **credit_score** - 
* **country** - 
* **gender** - 'Male' or 'Female'
* **age** - 
* **tenure** - assuming years with as customer
* **balance** - 
* **products_number** - assuming number of bank products used (e.g. credit, home loan, checking acct., etc.)
* **credit_card** - 1 or 0
* **active_member** - 1 or 0
* **estimated_salary** - 
* **churn** - used as the target. 1 if the client has left the bank during some period or 0 if he/she has not.

## Imports

In [2]:
# Analysis, Visualization
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# DBMS
import duckdb
# ML
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import (train_test_split, GridSearchCV,
                                     RandomizedSearchCV)
# Models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# Metrics
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score, 
accuracy_score, precision_score, recall_score, classification_report,
ConfusionMatrixDisplay)

## Database Creation, Loading Data

In [3]:
# creating connection to database
conn = duckdb.connect('churn_data.db')

In [6]:
# creating a 
df = conn.execute('''
select *
from 'Bank Customer Churn Prediction.csv'
''').df()

## Viewing Data

In [7]:
df.sample(5)

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
766,15810864,700,France,Female,82,2,0.00,2,0,1,182055.36,0
1164,15750776,850,France,Female,36,0,164850.54,1,1,1,62722.44,0
8465,15601569,598,France,Female,40,2,171178.25,1,1,0,137980.58,1
5926,15757559,595,France,Female,53,7,0.00,2,1,0,41371.68,1
4537,15777436,710,Spain,Female,31,5,0.00,2,1,0,9561.73,0


In [9]:
display(df.describe(include='all').T, df.info(), df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
customer_id,10000.0,NaN,NaN,NaN,15690940.5694,71936.186123,15565701.0,15628528.25,15690738.0,15753233.75,15815690.0
credit_score,10000.0,NaN,NaN,NaN,650.5288,96.653299,350.0,584.0,652.0,718.0,850.0
country,10000,3,France,5014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gender,10000,2,Male,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,10000.0,NaN,NaN,NaN,38.9218,10.487806,18.0,32.0,37.0,44.0,92.0
tenure,10000.0,NaN,NaN,NaN,5.0128,2.892174,0.0,3.0,5.0,7.0,10.0
balance,10000.0,NaN,NaN,NaN,76485.889288,62397.405202,0.0,0.0,97198.54,127644.24,250898.09
products_number,10000.0,NaN,NaN,NaN,1.5302,0.581654,1.0,1.0,1.0,2.0,4.0
credit_card,10000.0,NaN,NaN,NaN,0.7055,0.45584,0.0,0.0,1.0,1.0,1.0
active_member,10000.0,NaN,NaN,NaN,0.5151,0.499797,0.0,0.0,1.0,1.0,1.0


None

Index(['customer_id', 'credit_score', 'country', 'gender', 'age', 'tenure',
       'balance', 'products_number', 'credit_card', 'active_member',
       'estimated_salary', 'churn'],
      dtype='object')

<mark><u>**Comment:**</u>

* <font color='dodgerblue' size=4><i>
    Overall this data looks pretty clean, we will still check for duplicates and NaNs, but it appears to have been pre-cleaned.
* <font color='dodgerblue' size=4><i>
    We should be able to discard customer id, but will check to make sure there are no duplicate values (customer entered in twice).
* <font color='dodgerblue' size=4><i>
    There are only 3 countries in the dataset, with France being the most frequent.  We'll have to keep this in mind when thinking of the scope of the model's usability
* <font color='dodgerblue' size=4><i>
    Most of our values are already ordinal/ohe encoded, but will have to adjust the 'gender' column.
    
</i></font>

## Cleaning

### checking for duplicates, NaNs

In [12]:
df.duplicated().sum()

0

In [14]:
df.isna().sum()

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

In [16]:
# checking customer id column to see if there are repeated values
df['customer_id'].value_counts().max()

1

### changing 'gender' to numerical, 1=male, 0=female

In [17]:
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})

# confirming
df['gender'].value_counts()

gender
1    5457
0    4543
Name: count, dtype: int64

### checking categorical values for errors

In [19]:
for i in df.columns:
    if df[i].dtype == 'object' or df[i].nunique() <= 10:
        print(i.upper(), '\n', df[i].unique(), df[i].dtype)

COUNTRY 
 ['France' 'Spain' 'Germany'] object
GENDER 
 [0 1] int64
PRODUCTS_NUMBER 
 [1 3 2 4] int64
CREDIT_CARD 
 [1 0] int64
ACTIVE_MEMBER 
 [1 0] int64
CHURN 
 [1 0] int64


<mark><u>**Comment:**</u>

<font color='dodgerblue' size=4><i>
All looks good!
</i></font>

### rechecking dtypes

In [20]:
df.dtypes

customer_id           int64
credit_score          int64
country              object
gender                int64
age                   int64
tenure                int64
balance             float64
products_number       int64
credit_card           int64
active_member         int64
estimated_salary    float64
churn                 int64
dtype: object